In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import math
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression
import warnings
import pdb
warnings.filterwarnings('ignore')
import random
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold

In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)  # 禁止hash随机化

set_seed(42)


In [3]:
# label: '推料器自动指令'
# train_data_path = './data/train/'
# test_data_path = './data/test/'
#
# train_cols = ['推料器自动指令', 'CO含量', 'HCL含量', 'NOx含量', 'SO2含量', '二次风调门', '二次风量', '给水流量',
#         '炉排实际运行指令', '炉排自动投退信号', '汽包水位', '推料器启停', '推料器自动投退信号',
#         '氧量设定值', '一次风调门', '一次风量', '引风机转速', '主蒸汽流量']
#
# test_cols = ['CO含量', 'HCL含量', 'NOx含量', 'SO2含量', '二次风调门', '二次风量', '给水流量',
#         '炉排实际运行指令', '炉排自动投退信号', '汽包水位', '推料器启停', '推料器自动投退信号',
#         '氧量设定值', '一次风调门', '一次风量', '引风机转速', '主蒸汽流量']
#
# print(len(train_cols))
# print(len(test_cols))

In [4]:
# train_df = pd.DataFrame()
# train_df['时间'] = pd.read_csv(train_data_path+'CO含量.csv')['时间']
#
# for n in tqdm(train_cols):
#     train_df[n] = pd.read_csv(train_data_path+f'{n}.csv')[n]
#
#

In [5]:
# test_df = pd.DataFrame()
# test_df['时间'] = pd.read_csv(test_data_path+'CO含量.csv')['时间']
#
# for n in tqdm(test_cols):
#     test_df[n] = pd.read_csv(test_data_path+f'{n}.csv')[n]
#
#

In [6]:
# # 有重复的行，需要去重
# print('训练集中重复的行数：', train_df.duplicated().sum())
# train_df = train_df.drop_duplicates()
#
# print('测试集中重复的行数：', test_df.duplicated().sum())
# test_df = test_df.drop_duplicates()
#
#

In [7]:
# train_df.to_csv('./data/train.csv', index=False)
# test_df.to_csv('./data/test.csv', index=False)


In [8]:
train_df = pd.read_csv('./data/train.csv')
train_df = train_df.dropna()

test_df = pd.read_csv('./data/test.csv')
train_df.info()
test_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 141188 entries, 0 to 141187
Data columns (total 19 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   时间         141188 non-null  object 
 1   推料器自动指令    141188 non-null  float64
 2   CO含量       141188 non-null  float64
 3   HCL含量      141188 non-null  float64
 4   NOx含量      141188 non-null  float64
 5   SO2含量      141188 non-null  float64
 6   二次风调门      141188 non-null  float64
 7   二次风量       141188 non-null  float64
 8   给水流量       141188 non-null  float64
 9   炉排实际运行指令   141188 non-null  float64
 10  炉排自动投退信号   141188 non-null  object 
 11  汽包水位       141188 non-null  float64
 12  推料器启停      141188 non-null  object 
 13  推料器自动投退信号  141188 non-null  object 
 14  氧量设定值      141188 non-null  float64
 15  一次风调门      141188 non-null  float64
 16  一次风量       141188 non-null  float64
 17  引风机转速      141188 non-null  float64
 18  主蒸汽流量      141188 non-null  float64
dtypes: float64(15), object(

In [9]:
data = pd.concat([train_df, test_df])

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142988 entries, 0 to 1799
Data columns (total 19 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   时间         142988 non-null  object 
 1   推料器自动指令    141188 non-null  float64
 2   CO含量       142988 non-null  float64
 3   HCL含量      142988 non-null  float64
 4   NOx含量      142988 non-null  float64
 5   SO2含量      142988 non-null  float64
 6   二次风调门      142988 non-null  float64
 7   二次风量       142988 non-null  float64
 8   给水流量       142988 non-null  float64
 9   炉排实际运行指令   142988 non-null  float64
 10  炉排自动投退信号   142988 non-null  object 
 11  汽包水位       142988 non-null  float64
 12  推料器启停      142988 non-null  object 
 13  推料器自动投退信号  142988 non-null  object 
 14  氧量设定值      142988 non-null  float64
 15  一次风调门      142988 non-null  float64
 16  一次风量       142988 non-null  float64
 17  引风机转速      142988 non-null  float64
 18  主蒸汽流量      142988 non-null  float64
dtypes: float64(15), object(4)

In [10]:
data['推料器启停'].value_counts()

True     102060
False     40928
Name: 推料器启停, dtype: int64

In [11]:
data = data[data['推料器启停']==1]
# data.info()

In [12]:
data = data.drop(columns=['推料器启停'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102060 entries, 169 to 1799
Data columns (total 18 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   时间         102060 non-null  object 
 1   推料器自动指令    100260 non-null  float64
 2   CO含量       102060 non-null  float64
 3   HCL含量      102060 non-null  float64
 4   NOx含量      102060 non-null  float64
 5   SO2含量      102060 non-null  float64
 6   二次风调门      102060 non-null  float64
 7   二次风量       102060 non-null  float64
 8   给水流量       102060 non-null  float64
 9   炉排实际运行指令   102060 non-null  float64
 10  炉排自动投退信号   102060 non-null  object 
 11  汽包水位       102060 non-null  float64
 12  推料器自动投退信号  102060 non-null  object 
 13  氧量设定值      102060 non-null  float64
 14  一次风调门      102060 non-null  float64
 15  一次风量       102060 non-null  float64
 16  引风机转速      102060 non-null  float64
 17  主蒸汽流量      102060 non-null  float64
dtypes: float64(15), object(3)
memory usage: 14.8+ MB


In [13]:
# data['gas'] = data['CO含量'] + data['HCL含量'] + data['NOx含量'] + data['SO2含量']
# # feat_list = ['二次风调门', '二次风量', '给水流量', '炉排实际运行指令', '汽包水位', '氧量设定值', '一次风调门', '一次风量', '引风机转速', '主蒸汽流量']
# feat_list = ['给水流量', '氧量设定值', '一次风量', '主蒸汽流量', '汽包水位' ]
# for f in tqdm(feat_list):
#     shift_f = []
#     shift_d = []
#     for i in range(200):
#         colname = f + '_shift_{}'.format(i + 1)
#         data[colname] = data[f].shift(i + 1)
#         shift_f.append(colname)
#
#         # # 与上n分钟的变化比例特征
#         # if i%60==0:
#         #     temp_name = f+'_ratio_{}'.format(i)
#         #     data[temp_name] = data[colname]/data[colname].shift(i+1)
#
#         # .diff用于计算一列中某元素与该列中前？个元素的差值（默认前一个元素）
#         colname = f + '_diff_{}'.format(i + 1)
#         data[colname] = data[f].diff(i + 1)
#         colname = f + '_diff_{}'.format(-(i + 1))
#         data[colname] = data[f].diff(-(i + 1))
#
#     # data[f+'_diff'] = data[f].diff(1)
#     # 对每一行：shift_列的值取平均，即将前n天的值取平均，取最大值，最最小值，最标准差
#
#     data[f + '_fore_steps_mean'] = data[shift_f].mean(1)
#     data[f + '_fore_steps_max'] = data[shift_f].max(1)
#     data[f + '_fore_steps_min'] = data[shift_f].min(1)
#     data[f + '_fore_steps_std'] = data[shift_f].std(1)
#     data[f + '_fore_steps_std'] = data[shift_f].skew(1)
#
#     data.drop(shift_f, axis=1, inplace=True)

In [14]:
#
data['炉排自动投退信号'] = data['炉排自动投退信号'].map(int)
# data['推料器启停'] = data['推料器启停'].map(int)
data['推料器自动投退信号'] = data['推料器自动投退信号'].map(int)

In [15]:
# groupby + shift(差值特征) + transform(count, mean, max, min, skew)

# def get_shift_feats(data, gap_list=[1], gp_col=[], target_col=''):
#     # gp_col可以是个列表进行多次分组，e.g. gp_col = [id1, id2]
#     for gap in gap_list:
#         # 后面减前面
#         data[''.join(gp_col)+f'{target_col}next{gap}'] = data.groupby(gp_col)[target_col].shift(-gap)
#         data[''.join(gp_col)+f'{target_col}next{gap}'] = data[''.join(gp_col)+f'{target_col}next{gap}'] - data[target_col]
# 
#         # 前面减后面
#         data[''.join(gp_col)+f'{target_col}prev{gap}'] = data.groupby(gp_col)[target_col].shift(+gap)
#         data[''.join(gp_col)+f'{target_col}prev{gap}'] = data[''.join(gp_col)+f'{target_col}next{gap}'] - data[target_col]
# 
# 
#         # 统计其不为nan的值
#         data[''.join(gp_col)+f'{target_col}next{gap}count'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}next{gap}'].transform('count')
#         data[''.join(gp_col)+f'{target_col}prev{gap}count'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}prev{gap}'].transform('count')
# 
#         # 统计其平均值
#         data[''.join(gp_col)+f'{target_col}next{gap}mean'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}next{gap}'].transform('mean')
#         data[''.join(gp_col)+f'{target_col}prev{gap}mean'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}prev{gap}'].transform('mean')
# 
#         # 统计其最大值
#         data[''.join(gp_col)+f'{target_col}next{gap}max'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}next{gap}'].transform('max')
#         data[''.join(gp_col)+f'{target_col}prev{gap}max'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}prev{gap}'].transform('max')
# 
#         # 统计其最小值
#         data[''.join(gp_col)+f'{target_col}next{gap}min'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}next{gap}'].transform('min')
#         data[''.join(gp_col)+f'{target_col}prev{gap}min'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}prev{gap}'].transform('min')
# 
#         # 统计其skew值
#         data[''.join(gp_col)+f'{target_col}next{gap}skew'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}next{gap}'].transform('skew')
#         data[''.join(gp_col)+f'{target_col}prev{gap}skew'] = data.groupby(gp_col)[''.join(gp_col)+f'{target_col}prev{gap}'].transform('skew')
# 
#     return data
# 

In [16]:

# gp_tg_cols = [(['推料器启停'], '给水流量'),
#               (['推料器启停'], '主蒸汽流量')]
# 
# 

In [17]:
# for col in tqdm(gp_tg_cols):
#     data = get_shift_feats(data, gap_list=list(range(1, 10)), gp_col=col[0], target_col=col[1])

In [18]:
# data.columns = [str(i) for i in data.columns]
# for i in data.columns:
#     print(i)

In [19]:
import featuretools as ft
label = data['推料器自动指令'].to_list()

data = data.drop(columns=['推料器自动指令'])

In [20]:
# Make an entityset and add the entity
es = ft.EntitySet(id = 'lj')
es.add_dataframe(dataframe_name = 'data', dataframe = data,
                         make_index = True, index = 'index')

# Run deep feature synthesis with transformation primitives
feature_matrix, feature_defs = ft.dfs(entityset = es, target_dataframe_name = 'data',
                                      trans_primitives = ['add_numeric', 'multiply_numeric'])

feature_matrix.head()


,CO含量,HCL含量,NOx含量,SO2含量,二次风调门,二次风量,给水流量,炉排实际运行指令,炉排自动投退信号,汽包水位,...,氧量设定值 * 汽包水位,氧量设定值 * 炉排实际运行指令,氧量设定值 * 炉排自动投退信号,氧量设定值 * 给水流量,汽包水位 * 炉排实际运行指令,汽包水位 * 炉排自动投退信号,汽包水位 * 给水流量,炉排实际运行指令 * 炉排自动投退信号,炉排实际运行指令 * 给水流量,炉排自动投退信号 * 给水流量
index,,,,,,,,,,,,,,,,,,,,,
0,0.8819,2.0926,62.1852,-0.0276,19.6134,4062.0190,74.1088,60.0,1,6.6555,...,39.9330,360.0,6.0,444.6528,399.330,6.6555,493.231118,60.0,4446.528,74.1088
1,1.3889,2.1111,62.1296,3.0000,19.6736,4062.0190,74.0785,60.0,1,6.5549,...,39.3294,360.0,6.0,444.4710,393.294,6.5549,485.577160,60.0,4444.710,74.0785
2,1.4167,2.0833,62.2037,3.0417,19.6111,4062.0190,74.0368,60.0,1,6.4923,...,38.9538,360.0,6.0,444.2208,389.538,6.4923,480.669117,60.0,4442.208,74.0368
3,1.4583,2.0417,62.1481,3.0833,19.6366,3774.9172,74.0325,60.0,1,6.5318,...,39.1908,360.0,6.0,444.1950,391.908,6.5318,483.565483,60.0,4441.950,74.0325
4,1.4375,2.0509,62.0926,3.0903,19.5972,4062.0190,74.0034,60.0,1,6.5143,...,39.0858,360.0,6.0,444.0204,390.858,6.5143,482.080349,60.0,4440.204,74.0034


In [21]:
feature_defs

[<Feature: CO含量>,
 <Feature: HCL含量>,
 <Feature: NOx含量>,
 <Feature: SO2含量>,
 <Feature: 二次风调门>,
 <Feature: 二次风量>,
 <Feature: 给水流量>,
 <Feature: 炉排实际运行指令>,
 <Feature: 炉排自动投退信号>,
 <Feature: 汽包水位>,
 <Feature: 推料器自动投退信号>,
 <Feature: 氧量设定值>,
 <Feature: 一次风调门>,
 <Feature: 一次风量>,
 <Feature: 引风机转速>,
 <Feature: 主蒸汽流量>,
 <Feature: CO含量 + HCL含量>,
 <Feature: CO含量 + NOx含量>,
 <Feature: CO含量 + SO2含量>,
 <Feature: CO含量 + 一次风调门>,
 <Feature: CO含量 + 一次风量>,
 <Feature: CO含量 + 主蒸汽流量>,
 <Feature: CO含量 + 二次风调门>,
 <Feature: CO含量 + 二次风量>,
 <Feature: CO含量 + 引风机转速>,
 <Feature: CO含量 + 推料器自动投退信号>,
 <Feature: CO含量 + 氧量设定值>,
 <Feature: CO含量 + 汽包水位>,
 <Feature: CO含量 + 炉排实际运行指令>,
 <Feature: CO含量 + 炉排自动投退信号>,
 <Feature: CO含量 + 给水流量>,
 <Feature: HCL含量 + NOx含量>,
 <Feature: HCL含量 + SO2含量>,
 <Feature: HCL含量 + 一次风调门>,
 <Feature: HCL含量 + 一次风量>,
 <Feature: HCL含量 + 主蒸汽流量>,
 <Feature: HCL含量 + 二次风调门>,
 <Feature: HCL含量 + 二次风量>,
 <Feature: HCL含量 + 引风机转速>,
 <Feature: HCL含量 + 推料器自动投退信号>,
 <Feature: HCL含量 + 氧量设定值>,
 <Feature: HCL含量 + 汽包水位

In [22]:
feature_matrix['推料器自动指令']  = label

In [23]:
# feature_matrix['推料器自动指令']
data = feature_matrix
data.head(5)

,CO含量,HCL含量,NOx含量,SO2含量,二次风调门,二次风量,给水流量,炉排实际运行指令,炉排自动投退信号,汽包水位,...,氧量设定值 * 炉排实际运行指令,氧量设定值 * 炉排自动投退信号,氧量设定值 * 给水流量,汽包水位 * 炉排实际运行指令,汽包水位 * 炉排自动投退信号,汽包水位 * 给水流量,炉排实际运行指令 * 炉排自动投退信号,炉排实际运行指令 * 给水流量,炉排自动投退信号 * 给水流量,推料器自动指令
index,,,,,,,,,,,,,,,,,,,,,
0,0.8819,2.0926,62.1852,-0.0276,19.6134,4062.0190,74.1088,60.0,1,6.6555,...,360.0,6.0,444.6528,399.330,6.6555,493.231118,60.0,4446.528,74.1088,60.0
1,1.3889,2.1111,62.1296,3.0000,19.6736,4062.0190,74.0785,60.0,1,6.5549,...,360.0,6.0,444.4710,393.294,6.5549,485.577160,60.0,4444.710,74.0785,60.0
2,1.4167,2.0833,62.2037,3.0417,19.6111,4062.0190,74.0368,60.0,1,6.4923,...,360.0,6.0,444.2208,389.538,6.4923,480.669117,60.0,4442.208,74.0368,60.0
3,1.4583,2.0417,62.1481,3.0833,19.6366,3774.9172,74.0325,60.0,1,6.5318,...,360.0,6.0,444.1950,391.908,6.5318,483.565483,60.0,4441.950,74.0325,60.0
4,1.4375,2.0509,62.0926,3.0903,19.5972,4062.0190,74.0034,60.0,1,6.5143,...,360.0,6.0,444.0204,390.858,6.5143,482.080349,60.0,4440.204,74.0034,60.0


In [24]:
# # 内存压缩
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
                start_mem - end_mem) / start_mem))
    return df

# 压缩使用内存
# 由于数据比较大，所以合理的压缩内存节省空间尤为的重要
# 使用reduce_mem_usage函数可以压缩近70%的内存占有。
data = reduce_mem_usage(data)

100%|██████████| 257/257 [00:08<00:00, 32.10it/s] 


Mem. usage decreased to 59.18 Mb (70.5% reduction)


In [25]:
print('nums features: ', len(data.columns))

nums features:  257


In [26]:
label = '推料器自动指令'
test_data = data.tail(1800)
train = data.iloc[:-1800, :]
train.reset_index(drop=True, inplace=True)
# features = train.columns.drop(['时间', label]).tolist()
features = train.columns.drop([label]).tolist()
train_x, train_y = train[features], train[label]

In [27]:
# train

In [28]:
# 交叉验证所使用的第三方库


kf = KFold(n_splits=5, shuffle=True, random_state=42)
test_x = test_data[features]
test_lgb = np.zeros(test_x.shape[0])



for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
    print('************************************  {} fold************************************'.format(str(i + 1)))
    trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], \
                                 train_x.iloc[valid_index], train_y[valid_index]

    dtrain = lgb.Dataset(trn_x, label=trn_y)
    dvalid = lgb.Dataset(val_x, label=val_y)

    watchlist = [dtrain, dvalid]

    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression_l1',
        'metric': 'l2_root',
        'learning_rate': 0.01,
        'reg_alpha': 0.7,
        'reg_lambda': 35,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'feature_fraction': 0.7,
        "random_seed": 1,
    }
    model = lgb.train(params, train_set=dtrain, num_boost_round=10000, early_stopping_rounds=1000, valid_sets=watchlist,
                      verbose_eval=200)

    test_pred = model.predict(test_x)
    test_lgb += test_pred / kf.n_splits

************************************  1 fold************************************
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034765 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 62193
[LightGBM] [Info] Number of data points in the train set: 80208, number of used features: 256
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 70.000000
Training until validation scores don't improve for 1000 rounds
[200]	training's rmse: 5.01915	valid_1's rmse: 4.96969
[400]	training's rmse: 4.51974	valid_1's rmse: 4.47068
[600]	training's rmse: 3.70375	valid_1's rmse: 3.65898
[800]	training's rmse: 3.1455	valid_1's rmse: 3.14431
[1000]	training's rmse: 2.8246	valid_1's rmse: 2.86111
[1200]	training's rmse: 2.65679	valid_1's rmse: 2.69711
[1400]	training's 

In [29]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
test_x = test_data[features]
test_xgb = np.zeros(test_x.shape[0])



for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
    print('************************************  {} fold************************************'.format(str(i + 1)))
    trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], \
                                 train_x.iloc[valid_index], train_y[valid_index]
    # max_depth=6,learning_rate=0.05,n_estimators=100,
    model = XGBRegressor(randam_state=42)
    model.fit(trn_x, trn_y,
          eval_set=[(val_x, val_y)],
          verbose=10, eval_metric='rmse')


    test_pred = model.predict(test_x)
    test_xgb += test_pred / kf.n_splits

************************************  1 fold************************************
[12:35:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "randam_state" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:48.40872
[10]	validation_0-rmse:2.69273
[20]	validation_0-rmse:1.74103
[30]	validation_0-rmse:1.48096
[40]	validation_0-rmse:1.32283
[50]	validation_0-rmse:1.21991
[60]	validation_0-rmse:1.14819
[70]	validation_0-rmse:1.10462
[80]	validation_0-rmse:1.05896
[90]	validation_0-rmse:1.02864
[99]	validation_0-rmse:1.00648
************************************  2 fold************************************
[12:36:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Par

In [30]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
test_x = test_data[features]
test_cab = np.zeros(test_x.shape[0])



for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
    print('************************************  {} fold************************************'.format(str(i + 1)))
    trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], \
                                 train_x.iloc[valid_index], train_y[valid_index]


    model = CatBoostRegressor(eval_metric='RMSE')
    model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
              metric_period=200,
              cat_features=[],
              use_best_model=True,
              verbose=1)



    test_pred = model.predict(test_x)
    test_cab += test_pred / kf.n_splits

************************************  1 fold************************************
Learning rate set to 0.101358
0:	learn: 10.2587328	test: 10.2232464	best: 10.2232464 (0)	total: 195ms	remaining: 3m 14s
200:	learn: 1.7102993	test: 1.7637921	best: 1.7637921 (200)	total: 5s	remaining: 19.9s
400:	learn: 1.1242875	test: 1.2498927	best: 1.2498927 (400)	total: 9.89s	remaining: 14.8s
600:	learn: 0.9041441	test: 1.0592025	best: 1.0592025 (600)	total: 14.7s	remaining: 9.77s
800:	learn: 0.7761361	test: 0.9577105	best: 0.9577105 (800)	total: 19.6s	remaining: 4.86s
999:	learn: 0.6902430	test: 0.8883395	best: 0.8883395 (999)	total: 24.6s	remaining: 0us

bestTest = 0.888339496
bestIteration = 999

************************************  2 fold************************************
Learning rate set to 0.101358
0:	learn: 10.2484442	test: 10.2640336	best: 10.2640336 (0)	total: 28.2ms	remaining: 28.2s
200:	learn: 1.7020533	test: 1.8129807	best: 1.8129807 (200)	total: 5.16s	remaining: 20.5s
400:	learn: 1.1485

In [31]:
pd.Series(test_lgb).describe()

count    1800.000000
mean       60.998414
std         5.295495
min        55.222541
25%        56.662194
50%        58.882301
75%        64.712533
max        78.536112
dtype: float64

In [32]:
pd.Series(test_xgb).describe()

count    1800.000000
mean       59.825250
std         4.733616
min        53.930469
25%        56.869467
50%        58.162006
75%        61.598067
max        80.009094
dtype: float64

In [33]:
pd.Series(test_cab).describe()

count    1800.000000
mean       60.631621
std         3.896469
min        55.942671
25%        57.845264
50%        58.666008
75%        63.428359
max        76.913838
dtype: float64

In [34]:
test_lxc = (test_lgb+test_xgb+test_cab)/3

In [35]:
pd.Series(test_lxc).describe()

count    1800.000000
mean       60.485095
std         4.549022
min        55.589250
25%        57.187316
50%        58.556414
75%        63.180976
max        78.311607
dtype: float64

In [37]:

# sub = test_data[['时间','推料器启停']]
# sub['lgb' + label] = test_lgb
# sub.reset_index(inplace=True)
# sub.columns = ['ID', 'Time', 'Aps', 'Ai']



sub = pd.DataFrame()
sub['ID'] = list(range(1,1801))
sub['Time'] = test_df['时间']
sub['Aps'] = test_df['推料器启停'].astype(bool)
sub['Ai'] = test_lxc

sub.to_csv('./lgb+xgb+cab.csv', index=False)